In [2]:
import pyodbc
import pymysql
import time
from datetime import datetime
import threading
import signal
import sys

# ================== تنظیمات ==================
INTERVAL_SECONDS = 60 * 60  # هر 5 دقیقه
RUNNING = True  # برای توقف با Ctrl+C

# ================== اتصالات ==================
def get_sql_connection():
    return pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=MKZ-DSAS\\DSAS;'
    'DATABASE=DSAS;'
    'UID=datadriven;'
    'PWD=5Rdx@4Rfv1355'
    )

def get_mysql_connection():
    return pymysql.connect(
        host="127.0.0.1", port=3306, user="root", password="", database="dsas",
        charset="utf8mb4", cursorclass=pymysql.cursors.DictCursor
    )

# ================== لیست AssetIDها ==================
asset_ids = [8341, 8342, 8343, 8344, 8346, 9286, 9287]
field_map = {8341: 'AssetID_8341', 8342: 'AssetID_8342', 8343: 'AssetID_8343',
             8344: 'AssetID_8344', 8346: 'AssetID_8346', 9286: 'AssetID_9286', 9287: 'AssetID_9287'}

# ================== تابع اصلی (همون کد قبلی) ==================
def run_task():
    print(f"\n{'='*60}")
    print(f"اجرای وظیفه: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*60}")

    conn_sql = None
    conn_mysql = None
    cursor = None

    try:
        # --- اتصال ---
        print("اتصال به SQL Server...")
        conn_sql = get_sql_connection()
        print("اتصال به MySQL...")
        conn_mysql = get_mysql_connection()

        # --- خواندن TimeStamps موجود ---
        print("خواندن TimeStamps موجود...")
        existing_ts = set()
        with conn_mysql.cursor() as c:
            c.execute("SELECT TimeStamps FROM main_table_mhi_lube_oil_11 WHERE unitID = 11")
            for row in c.fetchall():
                existing_ts.add(row['TimeStamps'])
        print(f"{len(existing_ts)} رکورد موجود است.")

        # --- خواندن 10 رکورد آخر برای 8341 ---
        print("خواندن 10 رکورد آخر برای AssetID=8341...")
        query_8341 = """
        SELECT TOP 2000 [Value], [RecordTime], [RecordDate], [DateTime], [TimeStamp]
        FROM [DSAS].[PDA].[Periodic_Values]
        WHERE [UnitID] = 11 AND [AssetID] = 8341
        ORDER BY [DateTime] DESC
        """
        cursor = conn_sql.cursor()
        cursor.execute(query_8341)
        rows_8341 = cursor.fetchall()

        if not rows_8341:
            print("هیچ داده‌ای برای AssetID=8341 پیدا نشد.")
            return

        inserted_count = 0
        for idx, row in enumerate(rows_8341):
            ts = row.TimeStamp
            if ts in existing_ts:
                print(f"رکورد {idx+1}: TimeStamps={ts} تکراری → رد شد.")
                continue

            dt = row.DateTime
            print(f"\n--- درج رکورد {idx+1} | TimeStamps={ts} ---")

            # درج اولیه
            insert_sql = """
            INSERT INTO main_table_mhi_lube_oil_11
            (unitID, AssetID_8341, RecordTime, RecordDate, DateTime, TimeStamps)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            try:
                with conn_mysql.cursor() as c:
                    c.execute(insert_sql, (11, row.Value, 
                                         dt.strftime('%H:%M:%S'), dt.strftime('%Y-%m-%d'), dt, ts))
                conn_mysql.commit()
                print("رکورد درج شد.")
                inserted_count += 1
            except Exception as e:
                print(f"خطا در درج: {e}")
                continue

            # آپدیت 6 فیلد دیگر
            for aid in [8342, 8343, 8344, 8346, 9286, 9287]:
                field = field_map[aid]
                near_sql = f"""
                SELECT TOP 1 [Value] FROM [DSAS].[PDA].[Periodic_Values]
                WHERE [UnitID]=11 AND [AssetID]=? AND ABS([TimeStamp] - ?) < 1000
                ORDER BY ABS([TimeStamp] - ?)
                """
                cursor.execute(near_sql, (aid, ts, ts))
                near_row = cursor.fetchone()

                if near_row and near_row.Value is not None:
                    value = float(near_row.Value)
                    source = "نزدیک"
                else:
                    avg_sql = f"""
                    SELECT AVG(CAST([Value] AS FLOAT)) FROM (
                        SELECT TOP 10 [Value] FROM [DSAS].[PDA].[Periodic_Values]
                        WHERE [UnitID]=11 AND [AssetID]=?
                        ORDER BY [DateTime] DESC
                    ) sub
                    """
                    cursor.execute(avg_sql, (aid,))
                    avg_row = cursor.fetchone()
                    value = round(float(avg_row[0]), 4) if avg_row and avg_row[0] else None
                    source = "میانگین"

                if value is not None:
                    update_sql = f"UPDATE main_table_mhi_lube_oil_11 SET {field} = %s WHERE TimeStamps = %s"
                    try:
                        with conn_mysql.cursor() as c:
                            c.execute(update_sql, (value, ts))
                        conn_mysql.commit()
                        print(f"   {field} = {value} ({source})")
                    except Exception as e:
                        print(f"   خطا در آپدیت {field}: {e}")
                else:
                    print(f"   {field}: داده‌ای پیدا نشد.")

        print(f"\nنتیجه این دور: {inserted_count} رکورد جدید درج و آپدیت شد.")

    except Exception as e:
        print(f"خطای کلی: {e}")
    finally:
        if cursor: cursor.close()
        if conn_sql: conn_sql.close()
        if conn_mysql: conn_mysql.close()

# ================== حلقه اصلی (هر 5 دقیقه) ==================
def scheduler():
    run_task()
    # while RUNNING:
    #     run_task()
    #     print(f"\nمنتظر {INTERVAL_SECONDS//60} دقیقه بعدی...")
    #     # خوابیدن با چک کردن هر ثانیه (برای Ctrl+C)
    #     for _ in range(INTERVAL_SECONDS):
    #         if not RUNNING:
    #             break
    #         time.sleep(1)

# ================== توقف با Ctrl+C ==================
def signal_handler(sig, frame):
    global RUNNING
    print('\nدریافت سیگنال توقف (Ctrl+C). در حال خروج...')
    RUNNING = False

signal.signal(signal.SIGINT, signal_handler)

# ================== اجرا ==================
if __name__ == "__main__":
    print("اسکریپت هر 60 دقیقه یکبار اجرا میشود...")
    print("برای توقف، Ctrl+C بزنید.")
    scheduler()
    print("اسکریپت با موفقیت متوقف شد.")

اسکریپت هر 60 دقیقه یکبار اجرا میشود...
برای توقف، Ctrl+C بزنید.

اجرای وظیفه: 2025-11-18 10:28:11
اتصال به SQL Server...
اتصال به MySQL...
خواندن TimeStamps موجود...
2789 رکورد موجود است.
خواندن 10 رکورد آخر برای AssetID=8341...
رکورد 1: TimeStamps=1763340043 تکراری → رد شد.
رکورد 2: TimeStamps=1763325657 تکراری → رد شد.
رکورد 3: TimeStamps=1763311996 تکراری → رد شد.
رکورد 4: TimeStamps=1763298469 تکراری → رد شد.
رکورد 5: TimeStamps=1763282014 تکراری → رد شد.
رکورد 6: TimeStamps=1763269510 تکراری → رد شد.
رکورد 7: TimeStamps=1763259119 تکراری → رد شد.
رکورد 8: TimeStamps=1763239246 تکراری → رد شد.
رکورد 9: TimeStamps=1763225924 تکراری → رد شد.
رکورد 10: TimeStamps=1763210597 تکراری → رد شد.
رکورد 11: TimeStamps=1763196598 تکراری → رد شد.
رکورد 12: TimeStamps=1763182451 تکراری → رد شد.
رکورد 13: TimeStamps=1763172992 تکراری → رد شد.
رکورد 14: TimeStamps=1763153044 تکراری → رد شد.
رکورد 15: TimeStamps=1763140569 تکراری → رد شد.
رکورد 16: TimeStamps=1763127443 تکراری → رد شد.
رکورد 17: T